In [ ]:
!pip install datasets

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load SST2 dataset
dataset = load_dataset("stanfordnlp/sst2")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class SST2Dataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(
            item['sentence'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(item['label'])
        }

# Prepare datasets
train_dataset = SST2Dataset(dataset['train'], tokenizer)
eval_dataset = SST2Dataset(dataset['validation'], tokenizer)
test_dataset = SST2Dataset(dataset['test'], tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 3
train_accuracies, eval_accuracies = [], []

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predictions = torch.argmax(outputs.logits, dim=-1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_accuracies.append(train_accuracy)

    # Evaluation
    model.eval()
    eval_correct = 0
    eval_total = 0

    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            eval_correct += (predictions == labels).sum().item()
            eval_total += labels.size(0)

    eval_accuracy = eval_correct / eval_total
    eval_accuracies.append(eval_accuracy)

    print(f"Epoch {epoch + 1}: Train Accuracy: {train_accuracy:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, num_epochs + 1), eval_accuracies, label="Eval Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Evaluation Accuracy")
plt.legend()
plt.savefig("accuracy_plot.png")
plt.close()

# Test set evaluation
best_epoch = eval_accuracies.index(max(eval_accuracies)) + 1
print(f"Best model found at epoch {best_epoch}")

model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_correct += (predictions == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkp

Epoch 1: Train Accuracy: 0.8696, Eval Accuracy: 0.9312


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  8.12it/s]


Epoch 2: Train Accuracy: 0.9420, Eval Accuracy: 0.9438


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  8.14it/s]


Epoch 3: Train Accuracy: 0.9562, Eval Accuracy: 0.9450
Best model found at epoch 3


Testing: 100%|██████████| 29/29 [00:03<00:00,  7.84it/s]

Test Accuracy: 0.0000


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.6 MB/s eta 0:00:00


imp

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from peft import get_peft_model, LoraConfig, TaskType

# Load SST2 dataset
dataset = load_dataset("stanfordnlp/sst2")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class SST2Dataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(
            item['sentence'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(item['label'])
        }

# Prepare datasets
train_dataset = SST2Dataset(dataset['train'], tokenizer)
eval_dataset = SST2Dataset(dataset['validation'], tokenizer)
test_dataset = SST2Dataset(dataset['test'], tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Model and LoRA configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 3
train_accuracies, eval_accuracies = [], []

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predictions = torch.argmax(outputs.logits, dim=-1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_accuracies.append(train_accuracy)

    # Evaluation
    model.eval()
    eval_correct = 0
    eval_total = 0

    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            eval_correct += (predictions == labels).sum().item()
            eval_total += labels.size(0)

    eval_accuracy = eval_correct / eval_total
    eval_accuracies.append(eval_accuracy)

    print(f"Epoch {epoch + 1}: Train Accuracy: {train_accuracy:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, num_epochs + 1), eval_accuracies, label="Eval Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Evaluation Accuracy (LoRA)")
plt.legend()
plt.savefig("accuracy_plot_lora.png")
plt.close()

# Test set evaluation
best_epoch = eval_accuracies.index(max(eval_accuracies)) + 1
print(f"Best model found at epoch {best_epoch}")

model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_correct += (predictions == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save the LoRA model
model.save_pretrained("roberta_sst2_lora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkp

trainable params: 1,034,498 || all params: 125,681,668 || trainable%: 0.8231


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Evaluating: 100%|██████████| 14/14 [00:01<00:00,  7.54it/s]


Epoch 1: Train Accuracy: 0.8931, Eval Accuracy: 0.9381


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  7.74it/s]


Epoch 2: Train Accuracy: 0.9215, Eval Accuracy: 0.9255


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  7.75it/s]


Epoch 3: Train Accuracy: 0.9246, Eval Accuracy: 0.9266
Best model found at epoch 1


Testing: 100%|██████████| 29/29 [00:03<00:00,  7.52it/s]


Test Accuracy: 0.0000


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot([log['step'] for log in trainer.state.log_history if 'loss' in log],
         [log['loss'] for log in trainer.state.log_history if 'loss' in log])
plt.title("Training Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.savefig("training_loss.png")
plt.show()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot([log['step'] for log in trainer.state.log_history if 'loss' in log],
         [log['loss'] for log in trainer.state.log_history if 'loss' in log])
plt.title("Training Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.savefig("training_loss.png")
plt.show()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from peft import get_peft_model, LoraConfig, TaskType

# Load SST2 dataset
dataset = load_dataset("stanfordnlp/sst2")

# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

# Define LoRA Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Get PEFT model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare dataset for training
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=1e-3,  # Higher learning rate for LoRA
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision training
)

# Define compute_metrics function for the Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Train the model
train_result = trainer.train()

# Evaluate on the validation set
val_results = trainer.evaluate()
print(f"Validation Accuracy: {val_results['eval_accuracy']:.4f}")

# Evaluate on the test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot([log['step'] for log in trainer.state.log_history if 'loss' in log],
         [log['loss'] for log in trainer.state.log_history if 'loss' in log])
plt.title("Training Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.savefig("training_loss.png")
plt.show()

# Save the LoRA model
model.save_pretrained("./lora_roberta_sst2")

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load SST2 dataset
dataset = load_dataset("stanfordnlp/sst2")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class SST2Dataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(
            item['sentence'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(item['label'])
        }

# Prepare datasets
train_dataset = SST2Dataset(dataset['train'], tokenizer)
eval_dataset = SST2Dataset(dataset['validation'], tokenizer)
test_dataset = SST2Dataset(dataset['test'], tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Model configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

# BitFit: Freeze all parameters except biases
for name, param in model.named_parameters():
    if 'bias' not in name:
        param.requires_grad = False

# Print trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")

model.to(device)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

# Training loop
num_epochs = 5  # Increased epochs for BitFit
train_accuracies, eval_accuracies = [], []

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predictions = torch.argmax(outputs.logits, dim=-1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_accuracies.append(train_accuracy)

    # Evaluation
    model.eval()
    eval_correct = 0
    eval_total = 0

    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            eval_correct += (predictions == labels).sum().item()
            eval_total += labels.size(0)

    eval_accuracy = eval_correct / eval_total
    eval_accuracies.append(eval_accuracy)

    print(f"Epoch {epoch + 1}: Train Accuracy: {train_accuracy:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, num_epochs + 1), eval_accuracies, label="Eval Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Evaluation Accuracy (BitFit)")
plt.legend()
plt.savefig("accuracy_plot_bitfit.png")
plt.close()

# Test set evaluation
best_epoch = eval_accuracies.index(max(eval_accuracies)) + 1
print(f"Best model found at epoch {best_epoch}")

model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_correct += (predictions == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save the BitFit model
torch.save(model.state_dict(), "roberta_sst2_bitfit.pth")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total parameters: 124647170
Trainable parameters: 102914
Percentage of trainable parameters: 0.08%


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  7.87it/s]


Epoch 1: Train Accuracy: 0.8667, Eval Accuracy: 0.9312


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  8.08it/s]


...

Epoch 4: Train Accuracy: 0.9085, Eval Accuracy: 0.9220


Evaluating: 100%|██████████| 14/14 [00:01<00:00,  8.07it/s]


Epoch 5: Train Accuracy: 0.9115, Eval Accuracy: 0.9209
Best model found at epoch 3


Testing: 100%|██████████| 29/29 [00:03<00:00,  7.77it/s]


Test Accuracy: 0.0000
